In [ ]:
from _init import *

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import random, torch, requests, json, time
from typing import List

from ranger.utils import json_utils
from ranger.corag.corag_result import QueryResult
from ranger.chain_generate.chain_generate_client import request_chain_generate, request_reset

In [ ]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    print(f'set_seed() seed : {seed}')

seed = 42
set_seed(seed)

In [ ]:
def datas_shuffle(datas: list, seed: int):
    rng = random.Random(seed)
    rng.shuffle(datas)


def load_datas(train_data_path: str, test_data_path: str, seed: int, do_print=False):
    train_datas = json_utils.load_file(train_data_path)
    test_datas = json_utils.load_file(test_data_path)
    datas_shuffle(train_datas, seed)
    datas_shuffle(test_datas, seed)
    
    return train_datas, test_datas

In [ ]:
work_dir = f'/home/nlpshlee/dev_env/git/repos/ranger'
data_dir = f'{work_dir}/data'
out_dir = f'{work_dir}/output'

train_data_path = f'{data_dir}/custom_musique_train_5000_final.jsonl'
test_data_path = f'{data_dir}/custom_multihopqa_eval_1000.jsonl'
train_datas, test_datas = load_datas(train_data_path, test_data_path, seed, do_print=False)

In [ ]:
# def request_chain_generate(datas, batch_size, n_chains, chain_depth, adapter_path=''):
#     url = CHAIN_SERVER_CONFIG['url_full_generate']
#     headers = {'Content-Type': 'application/json'}

#     # 전송할 데이터 구성
#     payload = {
#         'datas': datas,
#         'batch_size': batch_size,
#         'n_chains': n_chains,
#         'chain_depth': chain_depth,
#         'adapter_path': adapter_path
#     }
    
#     try:
#         print(f'# [ChainGenerateClient] Requesting chain generation for {len(datas)} items...')

#         # POST 요청 전송
#         start_time = time.time()
#         response = requests.post(url, headers=headers, data=json.dumps(payload))
#         elapsed = time.time() - start_time
        
#         if response.status_code == 200:
#             res_json = response.json()
#             print(f'# [ChainGenerateClient] Success! Received {res_json["count"]} results in {elapsed:.2f}s\n')
#             return res_json['results']
#         else:
#             print(f'# [ChainGenerateClient] Error generate : {response.status_code} - {response.text}\n')
#             return None

#     except requests.exceptions.ConnectionError:
#         print(f'# [ChainGenerateClient] Error : Connection failed. Is the server running at {CHAIN_SERVER_CONFIG["url"]} ?\n')
#         return None

In [ ]:
# def request_reset():
#     url = CHAIN_SERVER_CONFIG['url_full_reset']
#     headers = {'Content-Type': 'application/json'}

#     try:
#         print(f'# [ChainGenerateClient] Requesting chain reset...')

#         response = requests.post(url, headers=headers, data=[])

#         if response.status_code == 200:
#             print(f'# [ChainGenerateClient] Success! Chain generator reset complete.\n')
#             return True
#         else:
#             print(f'# [ChainGenerateClient] Error reset : {response.status_code} - {response.text}\n')
#             return False
    
#     except requests.exceptions.ConnectionError:
#         print(f'# [ChainGenerateClient] Error : Connection failed. Is the server running at {CHAIN_SERVER_CONFIG["url"]} ?\n')
#         return False

In [ ]:
# adapter_path = ''
adapter_path = '/home/nlpshlee/dev_env/git/repos/ranger/outputs/test/lora_adapter_2025-12-03-07-13-16/'

In [ ]:
request_reset()

responses: List[QueryResult] = request_chain_generate(train_datas[:10], 4, 5, 5)
responses_dict = [query_result.to_dict() for query_result in responses]
print(f'{json_utils.to_str(responses_dict)}')

In [ ]:
# from ranger.chain_generate.chain_generator import ChainGenerator

# chain_generator = ChainGenerator(VLLM_CONFIG, CORAG_CONFIG)
# vllm_engine = chain_generator._engine

# inputs = [
#     {"content": "You are using a search engine to answer the main query by iteratively searching the web. Given the following intermediate queries and answers, generate a new simple follow-up question that can help answer the main query. You may rephrase or decompose the main query when previous answers are not helpful. Ask simple follow-up questions only as the search engine may not understand complex questions.\n\n## Previous intermediate queries and answers\nNothing yet\n\n## Task description\nanswer multi-hop questions\n\n## Main query to answer\nWhy was the battle of the capital of the county having Hopewell Township important to the American revolution?\n\nRespond with a simple follow-up question that will help answer the main query, do not explain yourself or output anything else.",
#     "role": "user"},
#     {"content": "You are using a search engine to answer the main query by iteratively searching the web. Given the following intermediate queries and answers, generate a new simple follow-up question that can help answer the main query. You may rephrase or decompose the main query when previous answers are not helpful. Ask simple follow-up questions only as the search engine may not understand complex questions.\n\n## Previous intermediate queries and answers\nIntermediate query 1: What was the significance of Hopewell Township in the American Revolution?\nIntermediate answer 1: no relevant information found\n\n## Task description\nanswer multi-hop questions\n\n## Main query to answer\nWhy was the battle of the capital of the county having Hopewell Township important to the American revolution?\n\nRespond with a simple follow-up question that will help answer the main query, do not explain yourself or output anything else.",
#     "role": "user"},
#     {"content": "You are using a search engine to answer the main query by iteratively searching the web. Given the following intermediate queries and answers, generate a new simple follow-up question that can help answer the main query. You may rephrase or decompose the main query when previous answers are not helpful. Ask simple follow-up questions only as the search engine may not understand complex questions.\n\n## Previous intermediate queries and answers\nIntermediate query 1: What was the significance of Hopewell Township in the American Revolution?\nIntermediate answer 1: no relevant information found\nIntermediate query 2: What is the capital of the county where Hopewell Township is located?\nIntermediate answer 2: no relevant information found\n\n## Task description\nanswer multi-hop questions\n\n## Main query to answer\nWhy was the battle of the capital of the county having Hopewell Township important to the American revolution?\n\nRespond with a simple follow-up question that will help answer the main query, do not explain yourself or output anything else.",
#     "role": "user"},
#     {"content": "You are using a search engine to answer the main query by iteratively searching the web. Given the following intermediate queries and answers, generate a new simple follow-up question that can help answer the main query. You may rephrase or decompose the main query when previous answers are not helpful. Ask simple follow-up questions only as the search engine may not understand complex questions.\n\n## Previous intermediate queries and answers\nIntermediate query 1: What was the significance of Hopewell Township in the American Revolution?\nIntermediate answer 1: no relevant information found\nIntermediate query 2: What is the capital of the county where Hopewell Township is located?\nIntermediate answer 2: no relevant information found\nIntermediate query 3: What is the name of the county where Hopewell Township is located?\nIntermediate answer 3: beaver county bedford county cumberland county huntingdon county washington county york county seneca county ohio cumberland county new jersey mercer county muskingum county perry county licking county\n\n## Task description\nanswer multi-hop questions\n\n## Main query to answer\nWhy was the battle of the capital of the county having Hopewell Township important to the American revolution?\n\nRespond with a simple follow-up question that will help answer the main query, do not explain yourself or output anything else.",
#     "role": "user"},
#     {"content": "You are using a search engine to answer the main query by iteratively searching the web. Given the following intermediate queries and answers, generate a new simple follow-up question that can help answer the main query. You may rephrase or decompose the main query when previous answers are not helpful. Ask simple follow-up questions only as the search engine may not understand complex questions.\n\n## Previous intermediate queries and answers\nIntermediate query 1: What was the significance of Hopewell Township in the American Revolution?\nIntermediate answer 1: no relevant information found\nIntermediate query 2: What is the capital of the county where Hopewell Township is located?\nIntermediate answer 2: no relevant information found\nIntermediate query 3: What is the name of the county where Hopewell Township is located?\nIntermediate answer 3: beaver county bedford county cumberland county huntingdon county washington county york county seneca county ohio cumberland county new jersey mercer county muskingum county perry county licking county\nIntermediate query 4: What is the capital of the county where Hopewell Township is located?\nIntermediate answer 4: no relevant information found\n\n## Task description\nanswer multi-hop questions\n\n## Main query to answer\nWhy was the battle of the capital of the county having Hopewell Township important to the American revolution?\n\nRespond with a simple follow-up question that will help answer the main query, do not explain yourself or output anything else.",
#     "role": "user"}
# ]

# # 서브 쿼리 생성
# sub_querys = vllm_engine.generate_batch(
#     datas=inputs,
#     return_completion_output=False,
#     adapter_path=''
# )

# print(f'sub_querys :\n\n{sub_querys}')

In [ ]:
request_reset()

responses: List[QueryResult] = request_chain_generate(train_datas[:10], 4, 5, 5, adapter_path)
responses_dict = [query_result.to_dict() for query_result in responses]
print(f'{json_utils.to_str(responses_dict)}')

In [ ]:
# # 직렬화, 역직렬화 테스트
# from _init import *
# from ranger.utils import json_utils
# from ranger.corag.corag_result import QueryResult, ChainResult

# query_result = QueryResult()
# query_result._query_id = 'q1'
# query_result._query = 'query test 1'
# query_result._answers = ['answer1', 'answer2', 'answer1']
# query_result._answer_set = set(query_result._answers)
# query_result._hop = 2
# query_result._chain_results = []
# query_result._doc_ids = [3, 4, 7]
# query_result._docs = ['doc3', 'doc4', 'doc7']
# query_result._em = 0.7
# query_result._f1 = 0.3

# chain_result1 = ChainResult()
# chain_result1._sub_querys = ['sub_query1', 'sub_query2', 'sub_query3']
# chain_result1._sub_answers = ['sub_answer1', 'sub_answer2', 'sub_answer3']
# chain_result1._doc_ids_list = [[1,2,3], [4,5,6], [7,8,9]]
# chain_result1._docs_list = [['doc1', 'doc2', 'doc3'], ['doc4', 'doc5', 'doc6'], ['doc7', 'doc8', 'doc9']]
# chain_result1._final_answers = ['final_answer1', 'final_answer2', 'final_answer3']
# chain_result1._log_probs = [0.01, 0.02, 0.03]
# chain_result1._is_stop = False
# chain_result1._reward = 0.02
# chain_result1._advantage = 2
# chain_result1._completion = 'completion1'
# chain_result1._em = 0.11
# chain_result1._f1 = 0.22

# chain_result2 = ChainResult()
# chain_result2._sub_querys = ['sub_query4', 'sub_query5', 'sub_query6']
# chain_result2._sub_answers = ['sub_answer4', 'sub_answer5', 'sub_answer6']
# chain_result2._doc_ids_list = [[11,12,13], [14,15,16], [17,18,19]]
# chain_result2._docs_list = [['doc11', 'doc12', 'doc13'], ['doc14', 'doc15', 'doc16'], ['doc17', 'doc18', 'doc19']]
# chain_result2._final_answers = ['final_answer11', 'final_answer12', 'final_answer13']
# chain_result2._log_probs = [0.11, 0.12, 0.13]
# chain_result2._is_stop = True
# chain_result2._reward = 0.12
# chain_result2._advantage = -2
# chain_result2._completion = 'completion2'
# chain_result2._em = -0.11
# chain_result2._f1 = -0.22

# query_result._chain_results.append(chain_result1)
# query_result._chain_results.append(chain_result2)

In [ ]:
# print(f'{chain_result1}\n')
# print(f'{chain_result2}\n')
# print(f'{query_result}\n')
# print(f'{query_result._chain_results[0]}\n')
# print(f'{query_result._chain_results[1]}\n')

In [ ]:
# cr1_dict = chain_result1.to_dict()
# cr2_dict = chain_result2.to_dict()
# qr_dict = query_result.to_dict()

In [ ]:
# print(f'{json_utils.to_str(cr1_dict)}\n')
# print(f'{json_utils.to_str(cr2_dict)}\n')
# print(f'{json_utils.to_str(qr_dict)}\n')

In [ ]:
# chain_result1_ = ChainResult.from_dict(cr1_dict)
# chain_result2_ = ChainResult.from_dict(cr2_dict)
# query_result_ = QueryResult.from_dict(qr_dict)

In [ ]:
# print(f'{chain_result1_}\n')
# print(f'{chain_result2_}\n')
# print(f'{query_result_}\n')
# print(f'{query_result_._chain_results[0]}\n')
# print(f'{query_result_._chain_results[1]}\n')